# Scrape Airline Routes and Times from OTAs

@Nileka add a description here as to what this notebook offers

### Execute the cell below once to turn off debugging

In [1]:
'''
    WARNING CONTROL to display or ignore all warnings
'''
import warnings; warnings.simplefilter('ignore')     #switch betweeb 'default' and 'ignore'
import traceback

''' Set debug flag to view extended error messages; else set it to False to turn off debugging mode '''
debug = True

## Initialize the OTA scrape class

In [2]:
import os
import sys
from datetime import datetime, date, timedelta
#import configparser

ROOT_DIR = "/home/nuwan/workspace/rezgate/wrangler"
MODULE_DIR = os.path.join(ROOT_DIR, 'modules/ota/')
DATA_DIR = os.path.join(ROOT_DIR, 'data/transport/airlines/scraper/')
sys.path.insert(1,MODULE_DIR)
import otaUtils as otau
import airlineScrapers as airWS

if debug:
    import importlib
    airWS = importlib.reload(airWS)
    otau = importlib.reload(otau)
    
kwargs = {
    "ROOT_DIR":ROOT_DIR,   # absolute path to the wrangler dir
    "MODULE_DIR":MODULE_DIR,   # absolute path to the ota module
    "DATA_DIR":DATA_DIR, # absolute path to the scraper data dir
}
clsScraper = airWS.AirlineScraper(name="kayak and momondo ota scrapes",**kwargs)

All Utils software packages loaded successfully!
All AirlineScraper software packages loaded successfully!
All AirlineScraper software packages loaded successfully!
All Utils software packages loaded successfully!
Initialing Utils class for otaUtils with instance Inheriting Utilities class for airline itinerary scraping
Logging Utils info, warnings, and error to /home/nuwan/workspace/rezgate/wrangler/logs/modules/ota/app.log
Initialing AirlineScraper class for airlineScrapers with instance data
Logging AirlineScraper info, warnings, and error to /home/nuwan/workspace/rezgate/wrangler/logs/modules/ota/app.log
Data path set to /home/nuwan/workspace/rezgate/wrangler/data/transport/airlines/scraper/


## Initialize the set of OTA URL for Scraping

In [6]:
#path = DATA_PATH
file = "airOTAInputURLs.json"
start_date = date.today()
end_date = start_date + timedelta(days=1)
airportCodefile = "airportUSACities.csv"
filePath=os.path.join(DATA_DIR, airportCodefile)

try:
    if start_date < date.today():
        raise ValueError("Start date must be greater than today: %s" % str(date.today()))
    if end_date <= start_date:
        raise ValueError("End date %s is invalid. It must be greater than Start Date: %s" % (str(end_date),str(start_date)))

    kwargs = {
        "startDate": start_date,
        "endDate" : end_date,
#        'airportsFile' : airportCodefile,
        'codeColName' : 'airportCode',
        'departAirportCode': ['LAX','NYC','BOS'],
        'arriveAirportCode': ['LAS'],
    }

    _otaURLfilePath, _ota_url_parameterized_list  = clsScraper.build_scrape_url_info(
        file_name=file,
        dirPath=DATA_DIR,
        **kwargs,
    )
    if _otaURLfilePath:
        print("Data in: %s" % _otaURLfilePath)
    if len(_ota_url_parameterized_list)>0:
        print("Completed parameterizing urls with %d instances." 
              % (len(_ota_url_parameterized_list)))

except Exception as err:
    _s_fn_id = "Class <WebScraper> Function <read_folder_csv_to_df>"
    print("[Error]"+_s_fn_id, err)
    print(traceback.format_exc())


Loaded 2 airline agencies to begin scraping OTA data.
Processing kayak.com ...
['LAX', 'NYC', 'BOS'] ['LAS']
Processing momondo.com ...
['LAX', 'NYC', 'BOS'] ['LAS']
Data in: /home/nuwan/workspace/rezgate/wrangler/data/tmp/AirlineScraper-build-scrape-url-list.csv
Completed parameterizing urls with 6 instances.


## Prepare folder & file structure

In [97]:
from datetime import datetime, timezone

#dirPath = None
dirPath=os.path.join(DATA_PATH, "itinerary/")
_search_dt = datetime.now()
_search_dt = _search_dt + (datetime.min - _search_dt) % timedelta(minutes=15)
''' include the timezone '''
_search_dt = (_search_dt.replace(tzinfo=timezone.utc)).isoformat()
# kwargs = {'searchDateTime': _search_dt,
#           'storageLocation': "local",   # values can be "local" or "AWS_S3"
#          }
kwargs = {
#    'SEARCH_DATETIME': _search_dt,
    'PARENT_DIR': "itinerary/",
    'STORAGE_METHOD': "local",   # values can be "local" or "AWS_S3"
}

_current_search_data_dir = clsScraper.make_storage_dir(**kwargs)
print("Extracting data into %s for search datetime: %s" % (_current_search_data_dir,str(_search_dt)))

Extracting data into /home/nuwan/workspace/rezgate/wrangler/data/transport/airlines/scraper/itinerary/2022-9-25-22-30/ for search datetime: 2022-09-25T22:30:00+00:00


## Scrape data into CSVs
___TODO:___ pause if internet connection times out; might be possible with airflow.

In [100]:
import pandas as pd

if _otaURLfilePath:
    urlDF = pd.read_csv(_otaURLfilePath, sep=",")
    _otaURLParamDictList = urlDF.to_dict('records')

_l_saved_files = clsScraper.scrape_url_info(
    otaURLlist =_otaURLParamDictList,
#    otaURLfile = None,
    searchDT = _search_dt,
    data_store_dir =_current_search_data_dir)
print("Scraping completed and data saved in %s!" % _current_search_data_dir)

loading parameterized urls from list 80 records
[Error]function <_scrape_kayak_to_csv> No data received for https://www.kayak.com/flights/LAS-LAX/2022-09-25/1adults?a&fs=cfc=1;bfc=1;transportation=transportation_plane
Traceback (most recent call last):
  File "/home/nuwan/workspace/rezgate/wrangler/modules/ota/airlineScrapers.py", line 503, in _scrape_kayak_to_csv
    raise ValueError("No data received for %s" % (url))
ValueError: No data received for https://www.kayak.com/flights/LAS-LAX/2022-09-25/1adults?a&fs=cfc=1;bfc=1;transportation=transportation_plane

[Error]function <_scrape_kayak_to_csv> No data received for https://www.kayak.com/flights/LAS-NYC/2022-09-25/1adults?a&fs=cfc=1;bfc=1;transportation=transportation_plane
Traceback (most recent call last):
  File "/home/nuwan/workspace/rezgate/wrangler/modules/ota/airlineScrapers.py", line 503, in _scrape_kayak_to_csv
    raise ValueError("No data received for %s" % (url))
ValueError: No data received for https://www.kayak.com/fli

[Error]function <_scrape_kayak_to_csv> No data received for https://www.kayak.com/flights/LAS-SFO/2022-09-25/1adults?a&fs=cfc=1;bfc=1;transportation=transportation_plane
Traceback (most recent call last):
  File "/home/nuwan/workspace/rezgate/wrangler/modules/ota/airlineScrapers.py", line 503, in _scrape_kayak_to_csv
    raise ValueError("No data received for %s" % (url))
ValueError: No data received for https://www.kayak.com/flights/LAS-SFO/2022-09-25/1adults?a&fs=cfc=1;bfc=1;transportation=transportation_plane

[Error]function <_scrape_kayak_to_csv> No data received for https://www.kayak.com/flights/LAS-WAS/2022-09-25/1adults?a&fs=cfc=1;bfc=1;transportation=transportation_plane
Traceback (most recent call last):
  File "/home/nuwan/workspace/rezgate/wrangler/modules/ota/airlineScrapers.py", line 503, in _scrape_kayak_to_csv
    raise ValueError("No data received for %s" % (url))
ValueError: No data received for https://www.kayak.com/flights/LAS-WAS/2022-09-25/1adults?a&fs=cfc=1;bfc=1

[Error]function <_scrape_kayak_to_csv> No data received for https://www.kayak.com/flights/LAS-CMH/2022-09-25/1adults?a&fs=cfc=1;bfc=1;transportation=transportation_plane
Traceback (most recent call last):
  File "/home/nuwan/workspace/rezgate/wrangler/modules/ota/airlineScrapers.py", line 503, in _scrape_kayak_to_csv
    raise ValueError("No data received for %s" % (url))
ValueError: No data received for https://www.kayak.com/flights/LAS-CMH/2022-09-25/1adults?a&fs=cfc=1;bfc=1;transportation=transportation_plane

[Error]function <_scrape_kayak_to_csv> No data received for https://www.kayak.com/flights/LAS-IND/2022-09-25/1adults?a&fs=cfc=1;bfc=1;transportation=transportation_plane
Traceback (most recent call last):
  File "/home/nuwan/workspace/rezgate/wrangler/modules/ota/airlineScrapers.py", line 503, in _scrape_kayak_to_csv
    raise ValueError("No data received for %s" % (url))
ValueError: No data received for https://www.kayak.com/flights/LAS-IND/2022-09-25/1adults?a&fs=cfc=1;bfc=1

[Error]function <_scrape_kayak_to_csv> No data received for https://www.kayak.com/flights/LAS-DTT/2022-09-25/1adults?a&fs=cfc=1;bfc=1;transportation=transportation_plane
Traceback (most recent call last):
  File "/home/nuwan/workspace/rezgate/wrangler/modules/ota/airlineScrapers.py", line 503, in _scrape_kayak_to_csv
    raise ValueError("No data received for %s" % (url))
ValueError: No data received for https://www.kayak.com/flights/LAS-DTT/2022-09-25/1adults?a&fs=cfc=1;bfc=1;transportation=transportation_plane

[Error]function <_scrape_kayak_to_csv> No data received for https://www.kayak.com/flights/LAS-MEM/2022-09-25/1adults?a&fs=cfc=1;bfc=1;transportation=transportation_plane
Traceback (most recent call last):
  File "/home/nuwan/workspace/rezgate/wrangler/modules/ota/airlineScrapers.py", line 503, in _scrape_kayak_to_csv
    raise ValueError("No data received for %s" % (url))
ValueError: No data received for https://www.kayak.com/flights/LAS-MEM/2022-09-25/1adults?a&fs=cfc=1;bfc=1

[Error]function <_scrape_kayak_to_csv> No data received for https://www.kayak.com/flights/LAS-OAK/2022-09-25/1adults?a&fs=cfc=1;bfc=1;transportation=transportation_plane
Traceback (most recent call last):
  File "/home/nuwan/workspace/rezgate/wrangler/modules/ota/airlineScrapers.py", line 503, in _scrape_kayak_to_csv
    raise ValueError("No data received for %s" % (url))
ValueError: No data received for https://www.kayak.com/flights/LAS-OAK/2022-09-25/1adults?a&fs=cfc=1;bfc=1;transportation=transportation_plane

[Error]function <_scrape_kayak_to_csv> No data received for https://www.momondo.com/flight-search/LAS-LAX/2022-09-25?a&fs=cfc=1;bfc=1;transportation=transportation_plane
Traceback (most recent call last):
  File "/home/nuwan/workspace/rezgate/wrangler/modules/ota/airlineScrapers.py", line 503, in _scrape_kayak_to_csv
    raise ValueError("No data received for %s" % (url))
ValueError: No data received for https://www.momondo.com/flight-search/LAS-LAX/2022-09-25?a&fs=cfc=1;bfc=1

[Error]function <_scrape_kayak_to_csv> No data received for https://www.momondo.com/flight-search/LAS-HOU/2022-09-25?a&fs=cfc=1;bfc=1;transportation=transportation_plane
Traceback (most recent call last):
  File "/home/nuwan/workspace/rezgate/wrangler/modules/ota/airlineScrapers.py", line 503, in _scrape_kayak_to_csv
    raise ValueError("No data received for %s" % (url))
ValueError: No data received for https://www.momondo.com/flight-search/LAS-HOU/2022-09-25?a&fs=cfc=1;bfc=1;transportation=transportation_plane

[Error]function <_scrape_kayak_to_csv> No data received for https://www.momondo.com/flight-search/LAS-PHL/2022-09-25?a&fs=cfc=1;bfc=1;transportation=transportation_plane
Traceback (most recent call last):
  File "/home/nuwan/workspace/rezgate/wrangler/modules/ota/airlineScrapers.py", line 503, in _scrape_kayak_to_csv
    raise ValueError("No data received for %s" % (url))
ValueError: No data received for https://www.momondo.com/flight-search/LAS-PHL/2022-09-25?a&fs=cfc=1;bfc=1

[Error]function <_scrape_kayak_to_csv> No data received for https://www.momondo.com/flight-search/LAS-SEA/2022-09-25?a&fs=cfc=1;bfc=1;transportation=transportation_plane
Traceback (most recent call last):
  File "/home/nuwan/workspace/rezgate/wrangler/modules/ota/airlineScrapers.py", line 503, in _scrape_kayak_to_csv
    raise ValueError("No data received for %s" % (url))
ValueError: No data received for https://www.momondo.com/flight-search/LAS-SEA/2022-09-25?a&fs=cfc=1;bfc=1;transportation=transportation_plane

[Error]function <_scrape_kayak_to_csv> No data received for https://www.momondo.com/flight-search/LAS-DEN/2022-09-25?a&fs=cfc=1;bfc=1;transportation=transportation_plane
Traceback (most recent call last):
  File "/home/nuwan/workspace/rezgate/wrangler/modules/ota/airlineScrapers.py", line 503, in _scrape_kayak_to_csv
    raise ValueError("No data received for %s" % (url))
ValueError: No data received for https://www.momondo.com/flight-search/LAS-DEN/2022-09-25?a&fs=cfc=1;bfc=1

## Use Spark to Load Data into Database

### Declare sparkworkload class

In [6]:
import os
import sys
from datetime import datetime, date, timedelta

ROOT_DIR = "/home/nuwan/workspace/rezgate/wrangler/"
#UTILS_PATH = os.path.join(ROOT_DIR, 'utils/')
UTILS_PATH = "/home/nuwan/workspace/rezgate/utils/"
sys.path.insert(1, UTILS_PATH)
import sparkWorkLoads as spark

if debug:
    import importlib
    spark = importlib.reload(spark)

kwargs = {
    "ROOT_DIR":ROOT_DIR,
}

clsSparkWL = spark.SparkWorkLoads(desc="ota airline segments", **kwargs)
#_session = clsSparkWL.get_spark_session()

All packages in SparkWorkLoads loaded successfully!
22/09/26 08:10:19 WARN Utils: Your hostname, FarmRaiderTester resolves to a loopback address: 127.0.1.1; using 192.168.124.15 instead (on interface enp2s0)
22/09/26 08:10:19 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/09/26 08:10:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


Initialing SparkWorkLoads class for sparkWorkLoads with instance ota airline segments
Logging SparkWorkLoads info, warnings, and error to /home/nuwan/workspace/rezgate/wrangler/logs/app.log


### Read CSV into spark dataframe

In [288]:
#dirPath = _current_search_data_dir
dirPath = "/home/nuwan/workspace/rezgate/wrangler/data/transport/airlines/scraper/itinerary/2022-9-19-22-30/"
_search_sdf = clsSparkWL.read_csv_to_sdf(filesPath=dirPath)
_search_sdf = _search_sdf.distinct()
print("Spark loaded %d rows" % _search_sdf.count())

[Error]Class <SparkWorkLoads> Function <read_folder_csv_to_sdf> No data files found in director: /home/nuwan/workspace/rezgate/wrangler/data/transport/airlines/scraper/itinerary/2022-9-19-22-30/
Traceback (most recent call last):
  File "/home/nuwan/workspace/rezgate/wrangler/utils/sparkWorkLoads.py", line 387, in read_csv_to_sdf
    raise ValueError("No data files found in director: %s" % (filesPath))
ValueError: No data files found in director: /home/nuwan/workspace/rezgate/wrangler/data/transport/airlines/scraper/itinerary/2022-9-19-22-30/

Spark loaded 0 rows


### Extract flight itinerary info

In [277]:
from pyspark.sql.functions import substring,lit,col, trim
from pyspark.sql.types import StringType,BooleanType,DateType,DecimalType,FloatType, IntegerType,LongType, ShortType, TimestampType

#_search_sdf=_search_sdf.withColumn("currency", lit("US$"))
#_search_sdf=_search_sdf.withColumn('room_rate', substring('room_rate', 4,10))

''' reset data types to match table '''
_search_sdf = _search_sdf.withColumn("booking_price",col("booking_price").cast(FloatType())) \
    .withColumn("search_dt",col("search_dt").cast(DateType()))
#    .withColumn("depart_time",col("depart_time").cast(TimestampType())) \
#    .withColumn("arrive_time",col("arrive_time").cast(TimestampType())) \
''' drop redundant columns '''
columns_to_drop = ['flight_times','flight_type','booking_rate','airline']
_search_sdf=_search_sdf.drop(*columns_to_drop)

_search_sdf.printSchema()
_search_sdf.show(n=2, vertical=True, truncate=False)
print("Split and Extraction complete!")

root
 |-- search_dt: date (nullable = true)
 |-- depart_port_code: string (nullable = true)
 |-- arrive_port_code: string (nullable = true)
 |-- num_stops: string (nullable = true)
 |-- duration: string (nullable = true)
 |-- num_passengers: integer (nullable = true)
 |-- depart_time: timestamp (nullable = true)
 |-- arrive_time: timestamp (nullable = true)
 |-- booking_price: float (nullable = true)
 |-- currency: string (nullable = true)

-RECORD 0----------------------------------
 search_dt        | 2022-09-20             
 depart_port_code | LAX                    
 arrive_port_code | LAS                    
 num_stops        | nonstop                
 duration         | 1h 09m  LAS &dash; LAX 
 num_passengers   | 1                      
 depart_time      | 2022-09-19 09:21:00    
 arrive_time      | 2022-09-19 10:30:00    
 booking_price    | 629.0                  
 currency         | US$                    
-RECORD 1----------------------------------
 search_dt        | 2022-09

## Cleanup and Save data to table

In [278]:
import os
''' Get airport codes dictionary '''
airportCodefile = "airportUSACities.csv"
airportsPath = os.path.join(DATA_PATH,'airports/')
airports_sdf = clsSparkWL.read_csv_to_sdf(filesPath=airportsPath)
airports_sdf = airports_sdf.withColumn("airportCode", trim(airports_sdf.airportCode))

_search_sdf = _search_sdf.withColumn("arrive_port_code", trim(_search_sdf.arrive_port_code)) \
                        .withColumn("depart_port_code", trim(_search_sdf.depart_port_code))\

''' first do the departure codes and names '''
dep_airports_sdf = airports_sdf.selectExpr("city as depart_port_name", \
                                       "airportCode as depart_port_code", \
                                      )
''' Lookup & augment departure name '''
aug_depart_sdf = dep_airports_sdf.join(_search_sdf,"depart_port_code")

''' next do the same for arrival codes and names '''
arr_airports_sdf = airports_sdf.selectExpr("city as arrive_port_name", \
                                       "airportCode as arrive_port_code", \
                                      )
''' Lookup & augment arrival name '''
aug_arr_sdf = arr_airports_sdf.join(aug_depart_sdf,on="arrive_port_code")

# aug_arr_sdf = _search_sdf.join(arr_airports_sdf,
#                                     trim(arr_airports_sdf['arrive_port_code']) == trim(_search_sdf['arrive_port_code'])
#                        ,'inner')
#                                    ).select(_search_sdf['arrive_port_name'].alias('arrive_port_name')
#                    ,arr_airports_sdf['users_count'])

arr_airports_sdf.printSchema()
#destinations_sdf.show(n=2, vertical=True, truncate=False)
print("airports dictionary loarded!")

#aug_search_sdf = airports_sdf.join(_search_sdf,"arrive_port_code")
#aug_search_sdf.show(n=2, vertical=True, truncate=False)
print("Destination names augmented to dataframe!")

root
 |-- arrive_port_name: string (nullable = true)
 |-- arrive_port_code: string (nullable = true)

airports dictionary loarded!
Destination names augmented to dataframe!


In [274]:
arr_airports_sdf.show(n=2, vertical=True, truncate=False)
aug_arr_sdf.show(n=2, vertical=True, truncate=False)

-RECORD 0---------------------
 arrive_port_name | Las Vegas 
 arrive_port_code | LAX       
-RECORD 1---------------------
 arrive_port_name | New York  
 arrive_port_code | NYC       
only showing top 2 rows

(0 rows)



## Save cleaned SDF to Tmp File

In [279]:
_tmp_fname = clsSparkWL.save_sdf_to_csv(aug_depart_sdf)
print("%d rows saved to %s" % (aug_depart_sdf.count(), _tmp_fname))

37 rows saved to /home/nuwan/workspace/rezgate/wrangler/data/tmp/save_sdf_to_csv.csv


## Read CSV from Tmp and Save to DB Table

In [283]:
''' Save dataframe to table '''
_s_tbl_name = "ota_airline_routes"
_get_tmp_sdf = clsSparkWL.read_csv_to_sdf(filesPath=_tmp_fname)
_get_tmp_sdf.printSchema()
count = clsSparkWL.insert_sdf_into_table(save_sdf=_get_tmp_sdf, dbTable=_s_tbl_name)
print("%d Data saved to %s" % (count,_s_tbl_name))

root
 |-- depart_port_code: string (nullable = true)
 |-- depart_port_name: string (nullable = true)
 |-- search_dt: timestamp (nullable = true)
 |-- arrive_port_code: string (nullable = true)
 |-- num_stops: string (nullable = true)
 |-- duration: string (nullable = true)
 |-- num_passengers: integer (nullable = true)
 |-- depart_time: timestamp (nullable = true)
 |-- arrive_time: timestamp (nullable = true)
 |-- booking_price: double (nullable = true)
 |-- currency: string (nullable = true)

Wait a moment while we insert data int ota_airline_routes
Save to ota_airline_routes complete!
37 Data saved to ota_airline_routes


## Read data from table

In [286]:
_s_tbl_name = "ota_airline_routes"
data = clsSparkWL.get_data_from_table(dbTable=_s_tbl_name)
print("%d records loaded from %s" %(data.count(),_s_tbl_name))
data.select("*").distinct().sort("search_dt").show(3, vertical=True, truncate=False)

Wait a moment, retrieving data ...
Loading complete!
37 records loaded from ota_airline_routes
-RECORD 0--------------------------------------
 uuid             | null                       
 ota_name         | null                       
 search_dt        | 2022-09-20 00:00:00        
 agent_name       | null                       
 depart_port_name | Las Vegas                  
 depart_port_code | LAX                        
 arrive_port_name | null                       
 arrive_port_code | LAS                        
 depart_time      | 2022-09-19 19:43:00        
 arrive_time      | 2022-09-19 20:59:00        
 num_stops        | nonstop                    
 duration         | 1h 16m  LAS &dash; LAX     
 cabin_type       | null                       
 booking_price    | 389.0                      
 currency         | US$                        
 num_passengers   | 1                          
 airline_name     | null                       
 data_source      | null                 

In [115]:
_tmp_sdf = data.select("*").distinct().sort("search_dt")
_tmp_fname = clsSparkWL.save_sdf_to_csv(_tmp_sdf, filesPath='/tmp/ota_prices.csv')
print(_tmp_fname)

/tmp/ota_prices.csv
